<a href="https://colab.research.google.com/github/CASAttackZW2025/CAS502Project/blob/main/CAS502ProcessingV001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the required packages for Excel writing and discrete-event simulation. The Version numbers have not been solidified att.
!pip install XlsxWriter simpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import simpy
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import io
import unittest
import random
from google.colab import files
from collections import OrderedDict
import numpy as np
import datetime


In [3]:
def ingest_excel_to_dfs(file_path_or_buffer, dtype_mapping: dict = None) -> dict:
    """
    Reads an Excel file and returns a dictionary mapping each sheet name to a pandas DataFrame.
    If a dtype_mapping is provided for a sheet, it converts the columns to the specified data types.

    Parameters:
        file_path_or_buffer (str or file-like): Path to the Excel file or a file-like object.
        dtype_mapping (dict): A mapping of sheet names to a dict of column names and their desired types.

    Returns:
        dict: A dictionary where keys are sheet names and values are DataFrames.
    """
    excel_file = pd.ExcelFile(file_path_or_buffer)
    dfs = {}
    for sheet in excel_file.sheet_names:
        df = excel_file.parse(sheet)
        # If there are calculated values, they will be read in as the last computed value.
        # Apply the data type conversion if a mapping is provided for this sheet.
        if dtype_mapping and sheet in dtype_mapping:
            try:
                df = df.astype(dtype_mapping[sheet])
            except Exception as e:
                print(f"Warning: Could not convert types for sheet '{sheet}': {e}")
        dfs[sheet] = df
    return dfs

def create_table_mapping(dfs: dict) -> pd.DataFrame:
    """
    Creates a summary mapping DataFrame of the ingested tables.

    Each row in the mapping contains:
        - table_name: Name of the sheet/table.
        - num_columns: Number of columns in the table.
        - num_rows: Number of rows in the table.
        - columns: List of column names.

    Parameters:
        dfs (dict): A dictionary with sheet names as keys and DataFrames as values.

    Returns:
        pd.DataFrame: A DataFrame summarizing the ingested tables.
    """
    mapping = []
    for name, df in dfs.items():
        mapping.append({
            "table_name": name,
            "num_columns": df.shape[1],
            "num_rows": df.shape[0],
            "columns": list(df.columns)
        })
    return pd.DataFrame(mapping)


In [4]:
# Ingestion and Printing in Colab
# This block prompts the user to upload an Excel file and then processes each
# uploaded file. The file is expected to have multiple sheets, each representing a
# table with specific production system data. Use the intake form instuctions to
# learn how to fill out the intake form
uploaded = files.upload()

# For each uploaded file, we define sheet names and how they are mapped to their normal data types.
# This "dtype_map" ensures that each column is correctly set/mapped/related to the desired data type.
for fn in uploaded.keys():
    dtype_map = {
        'ProcessorTypeTbl': {
            'ProcessorTypeIndex': int,
            'ProcessorTypeID': str,
            'ProcessorType': str,
        },
        'LocationTbl': {
            'LocationIndex': int,
            'LocationID': str,
            'LocationName': str,
            'AreaDesc': str,
            'SQFT': int,
        },
        'TaskTbl': {
            'TaskIndex': int,
            'TaskID': str,
            'TasksDesc': str,
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'GenRateProcessTime': float,
            'StndrdRateProcessTime': float,
            'VarRateProcessTime': float,
            'LearnCurvePct': float,
            'LearnCurveMaxThreshold': float,
            'MinProcessTimePct': float,
            'MaxProcessTimePct': float,
            'YeildPct': float,
            'BatchSizeReq': float,
            'UnbatchingSize': float,
            'ComponentsGenerated': str,
        },
        # For TaskResourcesTbl, we initially read all columns as strings;
        # later, columns with default names will be renamed to a standard format.
        'TaskResourcesTbl': {col: str for col in []},
        'ResourceTbl': {
            'ResourceIndex': int,
            'ResourceIndex.1': str,
            'ResourceName': str,
            'ResourceType': str,
            'TotalResourceUnitsAvailable': int,
            'DownProbabilityPerUnit': int,
            'DownReplacementDelay': int,
            'Units': str,
        },
        'ProcessorTbl': {
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'ProcessorTypeIndex': int,
            'ProcessorTypeID': str,
            'ProcessorType': str,
            'NumberStations': int,
            'LocationIndex': int,
            'LocationID': str,
            'Location': str,
            'DownProabilityPerUnity': int,
            'DownDelay': int,
            'ProcessingUnits': str,
        },
        'StationTbl': {
            'StationIndex': int,
            'StationID': str,
            'StationDesc': str,
            'StationType': str,
            'StationPriorityMethod': str,
            'StationCapacity': int,
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'DownProabilityPerUnit': int,
            'DownDelay': int,
            'Units': str,
        },
        'ProductsTbl': {
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ConfigTbl': {
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ProcessUnitTbl': {
            'ProcessUnitIndex': int,
            'ProcessUnitID': str,
            'ProcessUnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ArrivalUnitTbl': {
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ComponentsTbl': {
            'ComponentIndex': int,
            'ComponentID': str,
            'ComponentDesc': str,
            'UnitType': str,
            'UnitIndex': int,
            'UnitID': str,
            'UnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        # For Product01, we later rename task columns to a more user-friendly format.
        'Product01': {
            'TaskPred': str,
            # The task columns will be renamed post ingestion.
        },
        'ArrivalDatesTbl': {
            'ArrivalDatesIndex': int,
            'ArrivalDatesID': str,
            'ArrivalDatesDesc': str,
            'ArrivalDate': 'datetime64[ns]',
            'ArrivalCount': int,
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ArrivalRatesTbl': {
            'ArrivalRatesIndex': int,
            'ArrivalRatesID': str,
            'ArrivalRatesDesc': str,
            'ArrivalValue': int,
            'ArrivalUnits': str,
            'ArrivalCount': int,
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'DeliveryRatesTbl': {
            'DeliveryRatesIndex': int,
            'DeliveryRatesID': str,
            'DeliveryRatesDesc': str,
            'DeliveryRateValue': int,
            'DeliveryUnits': str,
            'DeliveryCount': int,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'DeliveryDatesTbl': {
            'DeliveryRatesIndex': int,
            'DeliveryRatesID': str,
            'DeliveryRatesDesc': str,
            'DeliveryRateDate': 'datetime64[ns]',
            'DeliveryCount': int,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'PersonnelTbl': {
            'PersonnelIndex': int,
            'PersonnelID': str,
            'PersonnelDesc': str,
            'Npersonnel': int,
            'ProductionRate': int,
            'HourlyRate': int,
            'Capacity': int,
            'Schedual': str,
            'Shift': str,
            'HolidayCalendar': str,
            'HolidayDays': str,
            'PTOUtilization': float,
            'PTORatePerWeek': float,
            'PTOCalendar': str,
            'OnTimeStartReliability': int,
            'OnTimeStartFailureDelay': int,
            'OvertimeAllowance': int,
            'OvertimeRate': int,
        },
        'SchdPtrnTbl': {
            'SchdPtrnIndex': int,
            'SchdPtrnID': str,
            'SchdPtrnDesc': str,
            'ShiftStartDate': 'datetime64[ns]',
            'Day01': str,
            'Day02': str,
            'Day03': str,
            'Day04': str,
            'Day05': str,
            'Day06': str,
            'Day07': str,
        },
        'ShiftTbl': {
            'ShiftIndex': int,
            'ShiftID': str,
            'ShiftDesc': str,
            'ShiftStart': str,
            'ShiftEnd': str,
            'ShiftStartUnit': int,
            'ShiftEndUnit': int,
            'ShiftLength': int,
            'ShiftBreakStart': str,
            'ShiftBreakEnd': str,
            'ShiftBreakStartUnit': int,
            'ShiftBreakEndUnit': int,
            'BreakLength': int,
            'StartBuffer': int,
            'EndBuffer': int,
            'ProductionStartTimePart1': int,
            'ProductionEndTimePart1': int,
            'ProductionStartTimePart2': int,
            'ProductionEndTimePart2': int,
            'TotalStaffingTime': int,
            'BaseProductionTime': int,
        },
        'HolidayClndrTbl': {
            'Year01': int,
            'HolidayCalendar01': 'datetime64[ns]',
            'HolidayCalendarDay01': int,
        },
        'PTOTbl': {
            'PTOYear01': int,
            'PTOCalendar01': 'datetime64[ns]',
            'PTODay01': int,
            'PTODay01.1': int,
            'PTOYearlySum01': int,
        },
    }

    # Ingest the Excel file into a dictionary of DataFrames.
    # The function "ingest_excel_to_dfs" reads each sheet, applies the data type mapping,
    # and returns a dictionary with sheet names as keys.
    dfs = ingest_excel_to_dfs(fn, dtype_mapping=dtype_map)

    # Post-Ingestion Renaming Steps
    # These steps clean up default column names that arise during ingestion,
    # making the DataFrames more user-friendly and standardized.

    # For TaskResourcesTbl, rename any columns with default names (e.g., "Unnamed: ...")
    # to a standardized format "NResourceXX" (where XX is a two-digit number).
    if 'TaskResourcesTbl' in dfs:
        tr_df = dfs['TaskResourcesTbl']
        rename_dict = {}
        for col in tr_df.columns:
            if col.startswith("Unnamed:"):
                try:
                    num = int(col.split(":")[1].strip())
                except ValueError:
                    num = 0
                rename_dict[col] = f"NResource{num:02d}"
        tr_df.rename(columns=rename_dict, inplace=True)

    # For Product01, rename task columns to a clearer format.
    # For example, change "task001" to "Task I1", "task002" to "Task I2", etc.
    if 'Product01' in dfs:
        p1_df = dfs['Product01']
        rename_dict = {}
        for col in p1_df.columns:
            # Exclude the 'TaskPred' column from renaming.
            if col.lower().startswith("task") and col.lower() != "taskpred":
                try:
                    num = int(col[4:])  # Extract the numeric portion after "task"
                except ValueError:
                    continue
                rename_dict[col] = f"Task I{num}"
        p1_df.rename(columns=rename_dict, inplace=True)

    # Create a summary mapping of the ingested DataFrames using the helper function.
    # This mapping includes the sheet name, number of columns, number of rows, and the column names.
    mapping_df = create_table_mapping(dfs)

    # Print the high-level summary mapping of all tables.
    print("TABLE MAPPING SUMMARY:")
    print(mapping_df)

    # For each table, print detailed information (DataFrame info and descriptive statistics)
    # to help the user verify that data ingestion was successful and correct.
    for table_name, df in dfs.items():
        print("\n" + "="*50)
        print(f"Table: {table_name}")
        print("="*50)
        print("DataFrame Info:")
        df.info()
        print("\nDataFrame Summary (describe):")
        print(df.describe(include='all'))



Saving production_system_intake_Example01.xlsx to production_system_intake_Example01.xlsx
TABLE MAPPING SUMMARY:
          table_name  num_columns  num_rows  \
0   ProcessorTypeTbl            3        15   
1        LocationTbl            5         5   
2            TaskTbl           17        80   
3   TaskResourcesTbl           53        80   
4        ResourceTbl            8        29   
5       ProcessorTbl           13        11   
6         StationTbl           12        31   
7        ProductsTbl            3         1   
8          ConfigTbl            6         1   
9     ProcessUnitTbl            9         1   
10    ArrivalUnitTbl            9         1   
11     ComponentsTbl           13         2   
12         Product01           81        80   
13   ArrivalDatesTbl           11         1   
14   ArrivalRatesTbl           12         1   
15  DeliveryRatesTbl            9         1   
16  DeliveryDatesTbl            8         1   
17      PersonnelTbl           18        

In [6]:

# Object Classes

class ProductionTask:
    """
    Holds data for a single task from TaskTbl
    and references any custom logic from TaskResourcesTbl.
    """
    def __init__(self, row_dict, resources_required=None):
        """
        row_dict: Dict of columns from a row in TaskTbl
        resources_required: Dict describing required resource usage from TaskResourcesTbl
        """
        # For example, the Excel sheet should have a column named 'TaskID'
        self.task_id = row_dict['TaskID']
        self.task_desc = row_dict['TasksDesc']
        self.processor_id = row_dict['ProcessorID']

        # Process times/ratios
        self.gen_rate_time = row_dict['GenRateProcessTime']
        self.std_rate_time = row_dict['StndrdRateProcessTime']
        self.var_rate_time = row_dict['VarRateProcessTime']
        self.learn_curve_pct = row_dict['LearnCurvePct']
        self.learn_curve_max = row_dict['LearnCurveMaxThreshold']

        self.min_proc_pct = row_dict['MinProcessTimePct']
        self.max_proc_pct = row_dict['MaxProcessTimePct']
        self.yield_pct   = row_dict['YeildPct']

        self.batch_size_req  = row_dict['BatchSizeReq']
        self.unbatching_size = row_dict['UnbatchingSize']

        self.components_generated = row_dict['ComponentsGenerated']

        # Reference the resource usage from TaskResourcesTbl
        # e.g. resources_required = { "NResource01": 1.0, "NResource02": 2.0, ... }
        self.resources_required = resources_required or {}

    def __repr__(self):
        return f"<ProductionTask {self.task_id}: {self.task_desc}>"


class ResourceDefinition:
    """
    Holds data from ResourceTbl.
    """
    def __init__(self, row_dict):
        self.resource_id    = row_dict['ResourceIndex.1']
        self.resource_name  = row_dict['ResourceName']
        self.resource_type  = row_dict['ResourceType']
        self.total_units    = row_dict['TotalResourceUnitsAvailable']
        self.down_prob      = row_dict['DownProbabilityPerUnit']
        self.replacement_delay = row_dict['DownReplacementDelay']
        self.units          = row_dict['Units']

    def __repr__(self):
        return f"<Resource {self.resource_id}: {self.resource_name}>"


class ProcessorDefinition:
    """
    Holds data from ProcessorTbl.
    """
    def __init__(self, row_dict):
        self.processor_id = row_dict['ProcessorID']
        self.processor_desc = row_dict['ProcessorDesc']
        self.processor_type_id = row_dict['ProcessorTypeID']

        # capacity / number of stations, or references to stations
        self.num_stations = row_dict['NumberStations']

        # reliability
        self.down_probability = row_dict['DownProabilityPerUnity']
        self.down_delay       = row_dict['DownDelay']

        self.processing_units  = row_dict['ProcessingUnits']

        # location references
        self.location_id   = row_dict['LocationID']
        self.location_desc = row_dict['Location']

    def __repr__(self):
        return f"<Processor {self.processor_id}: {self.processor_desc}>"


class StationDefinition:
    """
    Holds data from StationTbl.
    """
    def __init__(self, row_dict):
        self.station_id   = row_dict['StationID']
        self.station_desc = row_dict['StationDesc']
        self.station_type = row_dict['StationType']
        self.priority_method = row_dict['StationPriorityMethod']

        self.station_capacity = row_dict['StationCapacity']

        # reliability
        self.down_probability = row_dict['DownProabilityPerUnit']
        self.down_delay       = row_dict['DownDelay']

        self.units = row_dict['Units']

        # references to parent processor
        self.processor_id   = row_dict['ProcessorID']
        self.processor_desc = row_dict['ProcessorDesc']

    def __repr__(self):
        return f"<Station {self.station_id}: {self.station_desc}>"


class ProductDefinition:
    """
    Holds data from ProductsTbl.
    """
    def __init__(self, row_dict):
        self.product_id   = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

    def __repr__(self):
        return f"<Product {self.product_id}: {self.product_desc}>"


# Placeholder for ConfigTbl
class ConfigDefinition:
    def __init__(self, row_dict):
        self.config_id = row_dict['ConfigID']
        self.config_desc = row_dict['ConfigDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

    def __repr__(self):
        return f"<Config {self.config_id}: {self.config_desc}>"


# Placeholder for ProcessUnitTbl
class ProcessUnitDefinition:
    def __init__(self, row_dict):
        self.process_unit_id = row_dict['ProcessUnitID']
        self.process_unit_desc = row_dict['ProcessUnitDesc']
        self.config_index = row_dict['ConfigIndex']
        self.config_id = row_dict['ConfigID']
        self.config_desc = row_dict['ConfigDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

    def __repr__(self):
        return f"<ProcessUnit {self.process_unit_id}: {self.process_unit_desc}>"


# Placeholder for ArrivalUnitTbl
class ArrivalUnitDefinition:
    def __init__(self, row_dict):
        self.arrival_unit_id = row_dict['ArrivalUnitID']
        self.arrival_unit_desc = row_dict['ArrivalUnitDesc']
        self.config_index = row_dict['ConfigIndex']
        self.config_id = row_dict['ConfigID']
        self.config_desc = row_dict['ConfigDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

    def __repr__(self):
        return f"<ArrivalUnit {self.arrival_unit_id}: {self.arrival_unit_desc}>"


# Placeholder for ComponentsTbl
class ComponentDefinition:
    def __init__(self, row_dict):
        self.component_id = row_dict['ComponentID']
        self.component_desc = row_dict['ComponentDesc']
        self.unit_type = row_dict['UnitType']
        self.unit_index = row_dict['UnitIndex']
        self.unit_id = row_dict['UnitID']
        self.unit_desc = row_dict['UnitDesc']
        self.config_index = row_dict['ConfigIndex']
        self.config_id = row_dict['ConfigID']
        self.config_desc = row_dict['ConfigDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

    def __repr__(self):
        return f"<Component {self.component_id}: {self.component_desc}>"


# Placeholders for arrival and delivery tables

class ArrivalDatesDefinition:
    def __init__(self, row_dict):
        self.arrival_dates_id = row_dict['ArrivalDatesID']
        self.arrival_dates_desc = row_dict['ArrivalDatesDesc']
        self.arrival_date = row_dict['ArrivalDate']
        self.arrival_count = row_dict['ArrivalCount']
        self.arrival_unit_index = row_dict['ArrivalUnitIndex']
        self.arrival_unit_id = row_dict['ArrivalUnitID']
        self.arrival_unit_desc = row_dict['ArrivalUnitDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

class ArrivalRatesDefinition:
    def __init__(self, row_dict):
        self.arrival_rates_id = row_dict['ArrivalRatesID']
        self.arrival_rates_desc = row_dict['ArrivalRatesDesc']
        self.arrival_value = row_dict['ArrivalValue']
        self.arrival_units = row_dict['ArrivalUnits']
        self.arrival_count = row_dict['ArrivalCount']
        self.arrival_unit_index = row_dict['ArrivalUnitIndex']
        self.arrival_unit_id = row_dict['ArrivalUnitID']
        self.arrival_unit_desc = row_dict['ArrivalUnitDesc']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

class DeliveryRatesDefinition:
    def __init__(self, row_dict):
        self.delivery_rates_id = row_dict['DeliveryRatesID']
        self.delivery_rates_desc = row_dict['DeliveryRatesDesc']
        self.delivery_rate_value = row_dict['DeliveryRateValue']
        self.delivery_units = row_dict['DeliveryUnits']
        self.delivery_count = row_dict['DeliveryCount']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']

class DeliveryDatesDefinition:
    def __init__(self, row_dict):
        self.delivery_rates_id = row_dict['DeliveryRatesID']
        self.delivery_rates_desc = row_dict['DeliveryRatesDesc']
        self.delivery_rate_date = row_dict['DeliveryRateDate']
        self.delivery_count = row_dict['DeliveryCount']
        self.product_index = row_dict['ProductIndex']
        self.product_id = row_dict['ProductID']
        self.product_desc = row_dict['ProductDesc']


# Personnel and Shift definitions (we already defined PersonnelDefinition and ShiftDefinition)

class PersonnelDefinition:
    def __init__(self, row_dict):
        self.personnel_id = row_dict['PersonnelID']
        self.personnel_desc = row_dict.get('PersonnelDesc', '')
        self.n_personnel  = row_dict['Npersonnel']
        self.production_rate = row_dict['ProductionRate']
        self.hourly_rate = row_dict['HourlyRate']
        self.capacity = row_dict['Capacity']
        self.schedual = row_dict['Schedual']
        self.shift = row_dict['Shift']
        self.holiday_calendar = row_dict['HolidayCalendar']
        self.holiday_days = row_dict['HolidayDays']
        self.pto_utilization = row_dict['PTOUtilization']
        self.pto_rate_per_week = row_dict['PTORatePerWeek']
        self.pto_calendar = row_dict['PTOCalendar']
        self.on_time_start_reliability = row_dict['OnTimeStartReliability']
        self.on_time_start_failure_delay = row_dict['OnTimeStartFailureDelay']
        self.overtime_allowance = row_dict['OvertimeAllowance']
        self.overtime_rate = row_dict['OvertimeRate']

    def __repr__(self):
        return f"<Personnel {self.personnel_id}: {self.personnel_desc}>"


class ShiftDefinition:
    def __init__(self, row_dict):
        self.shift_id    = row_dict['ShiftID']
        self.shift_desc  = row_dict['ShiftDesc']
        self.shift_start = row_dict['ShiftStart']
        self.shift_end   = row_dict['ShiftEnd']
        self.shift_start_unit = row_dict['ShiftStartUnit']
        self.shift_end_unit   = row_dict['ShiftEndUnit']
        self.shift_length     = row_dict['ShiftLength']
        self.shift_break_start = row_dict['ShiftBreakStart']
        self.shift_break_end   = row_dict['ShiftBreakEnd']
        self.shift_break_start_unit = row_dict['ShiftBreakStartUnit']
        self.shift_break_end_unit   = row_dict['ShiftBreakEndUnit']
        self.break_length = row_dict['BreakLength']
        self.start_buffer = row_dict['StartBuffer']
        self.end_buffer   = row_dict['EndBuffer']
        self.production_start_time_part1 = row_dict['ProductionStartTimePart1']
        self.production_end_time_part1   = row_dict['ProductionEndTimePart1']
        self.production_start_time_part2 = row_dict['ProductionStartTimePart2']
        self.production_end_time_part2   = row_dict['ProductionEndTimePart2']
        self.total_staffing_time = row_dict['TotalStaffingTime']
        self.base_production_time = row_dict['BaseProductionTime']

    def __repr__(self):
        return f"<Shift {self.shift_id}: {self.shift_desc}>"

# Placeholder classes for scheduling pattern, holiday calendar, and PTO

class SchdPtrnDefinition:
    def __init__(self, row_dict):
        self.schd_ptrn_id = row_dict['SchdPtrnID']
        self.schd_ptrn_desc = row_dict['SchdPtrnDesc']
        self.shift_start_date = row_dict['ShiftStartDate']
        # Assume columns Day01...Day07 exist
        self.days = [row_dict.get(f'Day{i}', None) for i in range(1,8)]
    def __repr__(self):
        return f"<SchdPtrn {self.schd_ptrn_id}: {self.schd_ptrn_desc}>"

class HolidayClndrDefinition:
    def __init__(self, row_dict):
        self.year = row_dict['Year01']
        self.holiday_calendar = row_dict['HolidayCalendar01']
        self.holiday_calendar_day = row_dict['HolidayCalendarDay01']
    def __repr__(self):
        return f"<HolidayClndr {self.year}>"

class PTODefinition:
    def __init__(self, row_dict):
        self.pto_year = row_dict['PTOYear01']
        self.pto_calendar = row_dict['PTOCalendar01']
        self.pto_day1 = row_dict['PTODay01']
        self.pto_day2 = row_dict['PTODay01.1']
        self.pto_yearly_sum = row_dict['PTOYearlySum01']
    def __repr__(self):
        return f"<PTO {self.pto_year}: {self.pto_yearly_sum}>"

# Helper Function to Build the Production System from DataFrames
def build_production_system(dfs):
    """
    Build a production system dictionary from a dictionary of DataFrames.
    Each key in the resulting dictionary holds a list of instantiated objects
    corresponding to each production table.
    """
    system = {}

    # --- Build tasks ---
    tasks = []
    # Assume TaskTbl exists in dfs
    task_df = dfs.get('TaskTbl', pd.DataFrame())
    # Also get TaskResourcesTbl if available
    task_resources_df = dfs.get('TaskResourcesTbl', pd.DataFrame())
    for idx, row in task_df.iterrows():
        # Here we assume TaskTbl has a column 'TaskID'. If not, adjust the name.
        task_id = row.get('TaskID', None)
        # Look for matching resources using a common key (e.g., TaskIndex)
        resources_required = {}
        if not task_resources_df.empty:
            # We assume that TaskResourcesTbl has a 'TaskIndex' column that matches TaskTbl's 'TaskIndex'
            matching_rows = task_resources_df[task_resources_df['TaskIndex'] == row['TaskIndex']]
            if not matching_rows.empty:
                # For each column that starts with "NResource", if value is non-null and > 0, store it.
                for col in matching_rows.columns:
                    if col.startswith("NResource"):
                        val = matching_rows.iloc[0][col]
                        if pd.notna(val) and val != 0:
                            resources_required[col] = val
        tasks.append(ProductionTask(row, resources_required))
    system['tasks'] = tasks

    # --- Build resources ---
    resources = []
    resource_df = dfs.get('ResourceTbl', pd.DataFrame())
    for idx, row in resource_df.iterrows():
        resources.append(ResourceDefinition(row))
    system['resources'] = resources

    # --- Build processors ---
    processors = []
    processor_df = dfs.get('ProcessorTbl', pd.DataFrame())
    for idx, row in processor_df.iterrows():
        processors.append(ProcessorDefinition(row))
    system['processors'] = processors

    # --- Build stations ---
    stations = []
    station_df = dfs.get('StationTbl', pd.DataFrame())
    for idx, row in station_df.iterrows():
        stations.append(StationDefinition(row))
    system['stations'] = stations

    # --- Build products ---
    products = []
    products_df = dfs.get('ProductsTbl', pd.DataFrame())
    for idx, row in products_df.iterrows():
        products.append(ProductDefinition(row))
    system['products'] = products

    # --- Build config (if available) ---
    if 'ConfigTbl' in dfs:
        configs = []
        config_df = dfs['ConfigTbl']
        for idx, row in config_df.iterrows():
            configs.append(ConfigDefinition(row))
        system['configs'] = configs

    # --- Build process units ---
    if 'ProcessUnitTbl' in dfs:
        process_units = []
        pu_df = dfs['ProcessUnitTbl']
        for idx, row in pu_df.iterrows():
            process_units.append(ProcessUnitDefinition(row))
        system['process_units'] = process_units

    # --- Build arrival units ---
    if 'ArrivalUnitTbl' in dfs:
        arrival_units = []
        au_df = dfs['ArrivalUnitTbl']
        for idx, row in au_df.iterrows():
            arrival_units.append(ArrivalUnitDefinition(row))
        system['arrival_units'] = arrival_units

    # --- Build components ---
    if 'ComponentsTbl' in dfs:
        components = []
        comp_df = dfs['ComponentsTbl']
        for idx, row in comp_df.iterrows():
            components.append(ComponentDefinition(row))
        system['components'] = components

    # --- Build arrival dates (if applicable) ---
    if 'ArrivalDatesTbl' in dfs:
        arrival_dates = []
        ad_df = dfs['ArrivalDatesTbl']
        for idx, row in ad_df.iterrows():
            arrival_dates.append(ArrivalDatesDefinition(row))
        system['arrival_dates'] = arrival_dates

    # --- Build arrival rates (if applicable) ---
    if 'ArrivalRatesTbl' in dfs:
        arrival_rates = []
        ar_df = dfs['ArrivalRatesTbl']
        for idx, row in ar_df.iterrows():
            arrival_rates.append(ArrivalRatesDefinition(row))
        system['arrival_rates'] = arrival_rates

    # --- Build delivery rates (if applicable) ---
    if 'DeliveryRatesTbl' in dfs:
        delivery_rates = []
        dr_df = dfs['DeliveryRatesTbl']
        for idx, row in dr_df.iterrows():
            delivery_rates.append(DeliveryRatesDefinition(row))
        system['delivery_rates'] = delivery_rates

    # --- Build delivery dates (if applicable) ---
    if 'DeliveryDatesTbl' in dfs:
        delivery_dates = []
        dd_df = dfs['DeliveryDatesTbl']
        for idx, row in dd_df.iterrows():
            delivery_dates.append(DeliveryDatesDefinition(row))
        system['delivery_dates'] = delivery_dates

    # --- Build personnel ---
    if 'PersonnelTbl' in dfs:
        personnel = []
        pers_df = dfs['PersonnelTbl']
        for idx, row in pers_df.iterrows():
            personnel.append(PersonnelDefinition(row))
        system['personnel'] = personnel

    # --- Build shift definitions ---
    if 'ShiftTbl' in dfs:
        shifts = []
        shift_df = dfs['ShiftTbl']
        for idx, row in shift_df.iterrows():
            shifts.append(ShiftDefinition(row))
        system['shifts'] = shifts

    # --- Build scheduling patterns ---
    if 'SchdPtrnTbl' in dfs:
        schd_ptrns = []
        sp_df = dfs['SchdPtrnTbl']
        for idx, row in sp_df.iterrows():
            schd_ptrns.append(SchdPtrnDefinition(row))
        system['schedule_patterns'] = schd_ptrns

    # --- Build holiday calendars ---
    if 'HolidayClndrTbl' in dfs:
        holiday_calendars = []
        hc_df = dfs['HolidayClndrTbl']
        for idx, row in hc_df.iterrows():
            holiday_calendars.append(HolidayClndrDefinition(row))
        system['holiday_calendars'] = holiday_calendars

    # --- Build PTO definitions ---
    if 'PTOTbl' in dfs:
        pto_definitions = []
        pto_df = dfs['PTOTbl']
        for idx, row in pto_df.iterrows():
            pto_definitions.append(PTODefinition(row))
        system['pto'] = pto_definitions

    return system


if __name__ == '__main__':
    # filename = 'production_system_intake_Example01.xlsx'
    # dfs = pd.read_excel(filename, sheet_name=None)
    # For this example, we create a minimal dummy dfs dictionary.
    # Create a dummy TaskTbl DataFrame
    task_data = {
        'TaskIndex': [0],
        'TaskID': ['task001'],
        'TasksDesc': ['Pre-Prop subsystem'],
        'ProcessorIndex': [4],
        'ProcessorID': ['Processor04'],
        'ProcessorDesc': ['PY_Pannel_Integration'],
        'GenRateProcessTime': [18.16],
        'StndrdRateProcessTime': [14.91],
        'VarRateProcessTime': [5.40],
        'LearnCurvePct': [0.86],
        'LearnCurveMaxThreshold': [0.85],
        'MinProcessTimePct': [17.87],
        'MaxProcessTimePct': [25.82],
        'YeildPct': [0.999],
        'BatchSizeReq': [4.0],
        'UnbatchingSize': [6.0],
        'ComponentsGenerated': ['ComponentA']
    }
    task_df = pd.DataFrame(task_data)

    # Create a dummy TaskResourcesTbl DataFrame
    task_resources_data = {
        'TaskIndex': [0],
        'NResource01': [1.0],
        'NResource02': [2.0],
        # Assume many columns exist; here we include only two for brevity.
    }
    task_resources_df = pd.DataFrame(task_resources_data)

    # Create a dummy ResourceTbl DataFrame
    resource_data = {
        'ResourceIndex': [0],
        'ResourceIndex.1': ['Resource01'],
        'ResourceName': ['Egnr'],
        'ResourceType': ['EGSE'],
        'TotalResourceUnitsAvailable': [10],
        'DownProbabilityPerUnit': [0],
        'DownReplacementDelay': [0],
        'Units': ['hours']
    }
    resource_df = pd.DataFrame(resource_data)

    # Create dummy DataFrames for ProcessorTbl and StationTbl
    processor_data = {
        'ProcessorIndex': [4],
        'ProcessorID': ['Processor04'],
        'ProcessorDesc': ['PY_Pannel_Integration'],
        'ProcessorTypeIndex': [5],
        'ProcessorTypeID': ['PTYPE05'],
        'ProcessorType': ['NORMAL_PROCESSOR'],
        'NumberStations': [2],
        'LocationIndex': [2],
        'LocationID': ['LOC02'],
        'Location': ['Area02'],
        'DownProabilityPerUnity': [0],
        'DownDelay': [0],
        'ProcessingUnits': ['hours']
    }
    processor_df = pd.DataFrame(processor_data)

    station_data = {
        'StationIndex': [0],
        'StationID': ['Station01'],
        'StationDesc': ['REWORK'],
        'StationType': ['REWORK'],
        'StationPriorityMethod': ['FIFO'],
        'StationCapacity': [1],
        'ProcessorIndex': [4],
        'ProcessorID': ['Processor04'],
        'ProcessorDesc': ['PY_Pannel_Integration'],
        'DownProabilityPerUnit': [0],
        'DownDelay': [0],
        'Units': ['hours']
    }
    station_df = pd.DataFrame(station_data)

    # Create dummy DataFrames for ProductsTbl
    product_data = {
        'ProductIndex': [0],
        'ProductID': ['SAT01'],
        'ProductDesc': ['Satellite']
    }
    product_df = pd.DataFrame(product_data)

    # Create dummy DataFrames for PersonnelTbl and ShiftTbl if desired
    personnel_data = {
        'PersonnelIndex': [0],
        'PersonnelID': ['Personnel01'],
        'PersonnelDesc': ['TechShift01'],
        'Npersonnel': [8],
        'ProductionRate': [1],
        'HourlyRate': [1],
        'Capacity': [1],
        'Schedual': ['1stShiftSchedule'],
        'Shift': ['Shft01'],
        'HolidayCalendar': ['HolidayCalendar01'],
        'HolidayDays': ['Day01'],
        'PTOUtilization': [0.85],
        'PTORatePerWeek': [3.5],
        'PTOCalendar': ['PTOCalendar01'],
        'OnTimeStartReliability': [1],
        'OnTimeStartFailureDelay': [0],
        'OvertimeAllowance': [0],
        'OvertimeRate': [1]
    }
    personnel_df = pd.DataFrame(personnel_data)

    shift_data = {
        'ShiftIndex': [0],
        'ShiftID': ['Shft01'],
        'ShiftDesc': ['Day Shift'],
        'ShiftStart': ['08:00:00'],
        'ShiftEnd': ['16:00:00'],
        'ShiftStartUnit': [8],
        'ShiftEndUnit': [16],
        'ShiftLength': [8],
        'ShiftBreakStart': ['12:00:00'],
        'ShiftBreakEnd': ['12:30:00'],
        'ShiftBreakStartUnit': [12],
        'ShiftBreakEndUnit': [12],
        'BreakLength': [0.5],
        'StartBuffer': [0],
        'EndBuffer': [0],
        'ProductionStartTimePart1': [8],
        'ProductionEndTimePart1': [12],
        'ProductionStartTimePart2': [12.5],
        'ProductionEndTimePart2': [16],
        'TotalStaffingTime': [8],
        'BaseProductionTime': [6]
    }
    shift_df = pd.DataFrame(shift_data)

    # Build the dictionary of DataFrames as if read from Excel.
    dfs = {
        'TaskTbl': task_df,
        'TaskResourcesTbl': task_resources_df,
        'ResourceTbl': resource_df,
        'ProcessorTbl': processor_df,
        'StationTbl': station_df,
        'ProductsTbl': product_df,
        'PersonnelTbl': personnel_df,
        'ShiftTbl': shift_df
        # Additional sheets (ConfigTbl, ProcessUnitTbl, etc.) can be added here.
    }

    # Build the production system
    system_data = build_production_system(dfs)

    # Print out summaries for verification
    print("Tasks:")
    for t in system_data['tasks']:
        print(t)
    print("\nResources:")
    for r in system_data['resources']:
        print(r)
    print("\nProcessors:")
    for p in system_data['processors']:
        print(p)
    print("\nStations:")
    for s in system_data['stations']:
        print(s)
    print("\nProducts:")
    for prod in system_data['products']:
        print(prod)
    print("\nPersonnel:")
    for pers in system_data.get('personnel', []):
        print(pers)
    print("\nShifts:")
    for sh in system_data.get('shifts', []):
        print(sh)


Tasks:
<ProductionTask task001: Pre-Prop subsystem>

Resources:
<Resource Resource01: Egnr>

Processors:
<Processor Processor04: PY_Pannel_Integration>

Stations:
<Station Station01: REWORK>

Products:
<Product SAT01: Satellite>

Personnel:
<Personnel Personnel01: TechShift01>

Shifts:
<Shift Shft01: Day Shift>
